## MNIST
In this tutorial we will create a convolutional neural network that classifies handwritten digits (0-9) from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). The MNIST dataset has 60,000 training examples and 10,000 test samples. The digits are normalised for size and centred.

![MNIST Digits](https://camo.githubusercontent.com/d440ac2eee1cb3ea33340a2c5f6f15a0878e9275/687474703a2f2f692e7974696d672e636f6d2f76692f3051493378675875422d512f687164656661756c742e6a7067)

This is a multi-class classification problem. The main difference between the previous tutorials is that we are now processing images rather than a vector of numbers. Also, we are going to use the GPU to speed up computation.

### Import dependencies
Start by importing the dependencies we will need for the project

In [ ]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from uoa_mlaas import use_gpu
K.set_image_dim_ordering('th')
use_gpu()

### Set seed
Set a seed value so that when we repeatedly run our code we will get the same result. Using the same seed is important when you want to compare algorithms.

In [ ]:
seed = 7
numpy.random.seed(seed)

### Import data
The MNIST dataset has  60,000 training samples and 10,000 test samples.

Keras includes a number of datasets, including MNIST in the `keras.datasets` module. To download the MNIST dataset call `mnist.load_data()`.

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

A two dimensional convolutional neural network expects data to be arranged in a four dimensional shape: *number of samples* `x` *channels* `x` *width* `x` *height*.

For example, assume we have 10 RGB images, with widths and heights of 20 pixels. This would need to be shaped into 10 samples, each RGB channel would be split into three separate images (1 for red, 1 for green and 1 for blue). Each image would then be a 2D array with 20 rows and columns.

The MNIST data has the wrong shape for a 2D convolutional neural network (3 dimensions rather than four):

In [ ]:
print(X_train.shape)

The data is missing the dimension for channels, so we need to reshape the data to add it in. The images in the MNIST example are gray scale, so they only have 1 channel.

This is done using the numpy `reshape` method.

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

Normalize the pixels to the range 0 - 1

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

One hot encode the target variables as you did in the Iris classification tutorial. The data is already numeric so you do not need to use the LabelEncoder.

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
print("Number of classes: {0}".format(num_classes))

### Create the model

1. The input layer takes inputs with a dimension 1x28x28
2. The first hidden layer is a `Conv2D` layer. We have set it to have 30 filters (the number of output filters) and the size of the kernel to 5x5.
3. The `MaxPooling2D` has a kernel size of 2x2. This downsamples the output of the previous layer by selecting the maxiumum value in each kernel. An example is illustrated below: ![Max pooling](https://qph.ec.quoracdn.net/main-qimg-8afedfb2f82f279781bfefa269bc6a90.webp)
4. The `Dropout` layer is used to prevent overfitting. It does this by randomly turning off neurons (20% in this case).
5. A `Dense` fully connected layer is added with 50 neurons.
6. The last layer is the output layer, which has 10 neurons (one for each class/digit).

In [ ]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_initializer='normal'))
model.add(Dense(num_classes, activation='softmax', kernel_initializer='normal'))

### Compile the model
The next step is to compile the model. The loss function (`categorical_crossentropy`) is the same as the Iris multi-class classification tutorial.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the model
The next step is to train the model. It takes a lot more comuting resources to train convultional neural networks, so note that far less `epochs` are used, but a much larger `batch_size` is used due to a much larger data set.

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

### Evaluate the model
Now that we have trained our model, we can evaluate the performance on the test data.

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Error: {0:.2f}%".format((100-scores[1]*100)))